In [1]:
import pandas as pd

In [2]:
df_sc = pd.read_parquet("../data/sc_pre_processado.parquet")

In [3]:
df_vacinacoes = pd.read_parquet("../data/resultado.parquet")

In [4]:
df_sc.columns

Index(['document_id', 'paciente_id', 'paciente_idade',
       'paciente_dataNascimento', 'paciente_enumSexoBiologico',
       'paciente_racaCor_codigo', 'paciente_racaCor_valor',
       'paciente_endereco_coIbgeMunicipio', 'paciente_endereco_coPais',
       'paciente_endereco_nmMunicipio', 'paciente_endereco_nmPais',
       'paciente_endereco_uf', 'paciente_endereco_cep',
       'paciente_nacionalidade_enumNacionalidade', 'estabelecimento_valor',
       'estabelecimento_razaoSocial', 'estabelecimento_noFantasia',
       'estabelecimento_municipio_codigo', 'estabelecimento_municipio_nome',
       'estabelecimento_uf', 'vacina_grupoAtendimento_codigo',
       'vacina_grupoAtendimento_nome', 'vacina_categoria_codigo',
       'vacina_categoria_nome', 'vacina_lote', 'vacina_fabricante_nome',
       'vacina_fabricante_referencia', 'vacina_dataAplicacao',
       'vacina_descricao_dose', 'vacina_codigo', 'vacina_nome',
       'sistema_origem', 'idade_calculada', 'ano_aplicacao', 'mes_aplicacao

In [5]:
df_sc['paciente_endereco_nmMunicipio'].head()

index
0    PORTO UNIAO
1        BRUSQUE
2    BARRA VELHA
3       CAMBORIU
4          LAGES
Name: paciente_endereco_nmMunicipio, dtype: object

In [6]:
df_sc = df_sc.drop(
    columns=[
        'document_id', 'paciente_id', 'paciente_idade',
        'paciente_dataNascimento', 'paciente_enumSexoBiologico',
        'paciente_racaCor_codigo', 'paciente_racaCor_valor',
        'paciente_endereco_coIbgeMunicipio', 'paciente_endereco_coPais',
        'paciente_endereco_nmPais', 'paciente_endereco_uf', 'paciente_endereco_cep',
        'paciente_nacionalidade_enumNacionalidade', 'estabelecimento_valor',
        'estabelecimento_razaoSocial', 'estabelecimento_noFantasia',
        'estabelecimento_municipio_codigo', 'estabelecimento_municipio_nome',
        'estabelecimento_uf', 'vacina_grupoAtendimento_codigo',
        'vacina_grupoAtendimento_nome', 'vacina_categoria_codigo',
        'vacina_categoria_nome', 'vacina_lote', 'vacina_fabricante_nome',
        'vacina_fabricante_referencia', 'vacina_descricao_dose',
        'vacina_codigo', 'faixa_etaria', 'vacina_nome',
        'sistema_origem', 'idade_calculada', 'ano_aplicacao', 'mes_aplicacao'
    ]
)

In [7]:
df_sc = df_sc.rename(columns={'paciente_endereco_nmMunicipio': 'Município'})

In [8]:
df_sc['ano'] = df_sc['vacina_dataAplicacao'].dt.year

In [9]:
df_sc = df_sc.query('ano > 2020')

In [10]:
df_sc['ano'].value_counts()

ano
2021    11391882
2022     4045766
2024     2021634
2023      937894
Name: count, dtype: int64

In [11]:
df_teste = df_sc.groupby(['Município', 'ano']).size().reset_index(name='total')

In [12]:
df_pivot = df_teste.pivot_table(
    index="Município",
    columns="ano",
    values="total",
    aggfunc="sum"
)

In [13]:
df_pivot.head()

ano,2021,2022,2023,2024
Município,,,,
ABDON BATISTA,4735.0,2209.0,603.0,1230.0
ABELARDO LUZ,25556.0,9900.0,1833.0,4950.0
AGROLANDIA,16166.0,4985.0,870.0,2429.0
AGRONOMICA,10658.0,2703.0,526.0,1997.0
AGUA DOCE,11627.0,4129.0,888.0,2018.0


In [22]:
df_pivot.shape

(297, 4)

In [36]:
df_pivot.columns = df_pivot.columns.astype(str)

In [37]:
df_pivot = df_pivot.rename(columns={'2021': '2021_covid', '2022': '2022_covid', '2023': '2023_covid', '2024': '2024_covid'})

In [40]:
df_vacinacoes = df_vacinacoes.rename(
    columns={
        '2016': '2016_geral',
        '2017': '2017_geral',
        '2018': '2018_geral',
        '2019': '2019_geral',
        '2020': '2020_geral',
        '2021': '2021_geral',
        '2022': '2022_geral'
    }
)

In [44]:
df_final = df_vacinacoes.merge(
    df_pivot,
    how='inner',
    on='Município'
)

In [47]:
df_final.head()

ano,2016_geral,2017_geral,2018_geral,2019_geral,2020_geral,2021_geral,2022_geral,2021_covid,2022_covid,2023_covid,2024_covid
Município,,,,,,,,,,,
ARMAZEM,2539,3564,4609,8224,5295,4376,7497,14703.0,4663.0,898.0,3191.0
BALNEARIO CAMBORIU,34636,34293,58580,99544,80511,68340,64733,179664.0,58023.0,14047.0,27064.0
BALNEARIO PICARRAS,5784,12503,17308,19433,15633,12873,15074,38920.0,14070.0,3289.0,6810.0
BIGUACU,30165,25010,23772,64055,34121,37553,35446,97862.0,39842.0,11006.0,21720.0
BLUMENAU,175406,201927,236681,309696,228466,177062,172782,540094.0,171145.0,43051.0,116655.0


In [48]:
df_final.to_parquet("../data/df_final.parquet")